In [12]:
# Arun Venkitaraman 19August20
import math
import random
import time
import copy
import torch  # v0.4.1
from torch import nn
from torch.nn import functional as F
import matplotlib as mpl
import datetime
import dill


import gc 
gc.collect()

mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
use_cuda=True
nh =1 # no of neurons
kshot =4 # no fo shots or samples in each task
n_meta_train = 40000
n_tasks = 32 # no of training task
n_tasks_test= 64 # no of test tasks
reg =.01  # regularization 'mu' for gaussin kernel
reg_cos=.1  # regularization 'mu' for cosine kernel
sigma = 10
nx=16 # input dimension

alpha = 5e-2  # MAML grad step size
lr_maml=1e-2  #
lr_krmaml=1e-1  # Learning rate (LR) for Gaussian
lr_krmaml_cos=1e-2 # Learning rate for Cosine
lr_W=1e-3 # LR for regression coefficients for Gaussian kernel
lr_W_cos=1e-3 # LR for cosine

test_step=400

# comparing two kernels here
kernel_1 = 'gaussian'
kernel_2 = 'cosine'
fig_id='_02_10'
varying_amp=1
   # If 1 the amplitudes of task sinusoids vary, freq fixed
   # If 0, amp fixed, freq varies


n_rff=100

filename = 'global_' + fig_id+ '.pkl' #to save parameters
cmap = plt.get_cmap('jet')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def net(x, params):
    x = F.linear(x, params[0])
    return x


def kernel_product(w, x, mode="gaussian", s=0.1):
    w_i = w.unsqueeze(0)
    x_j = x.unsqueeze(0)
    xmy = ((w_i - x_j) ** 2).sum()
    # st()
    if mode == "gaussian":
        K = torch.exp(- (torch.t(xmy) ** 2) / (s ** 2))
    elif mode == "cosine":
        K = torch.dot(w, x) / (1e-7 + torch.norm(w, p=2) * torch.norm(x, p=2))
    elif mode == "laplace":
        K = torch.exp(- torch.sqrt(torch.t(xmy) + (s ** 2)))
    elif mode == "energy":
        K = torch.pow(torch.t(xmy) + (s ** 2), -.25)

    return K

def kernel_product_rff(w, x, mode="gaussian", s=0.1):
    w_i = w.unsqueeze(0)
    x_j = x.unsqueeze(0)
    xmy = ((w_i - x_j) ** 2).sum()
    

    # st()
    if mode == "gaussian":
        K = torch.exp(- (torch.t(xmy) ** 2) / (s ** 2))
    elif mode == "cosine":
        K = torch.dot(w, x) / (1e-7 + torch.norm(w, p=2) * torch.norm(x, p=2))
    elif mode == "laplace":
        K = torch.exp(- torch.sqrt(torch.t(xmy) + (s ** 2)))
    elif mode == "energy":
        K = torch.pow(torch.t(xmy) + (s ** 2), -.25)

    return K 

# square distance computation
def dist(x,y):
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)
    dist = torch.pow(x - y, 2).sum(2) 
    return dist

#Cosine distance matrix
def cos_dist(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a /(a_n+ eps * torch.ones_like(a_n))
    b_norm = b / (b_n+ eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.t())
    return sim_mt

params_maml = [
    torch.cuda.FloatTensor(nh, nx).uniform_(-1., 1.).requires_grad_(),

]


params_krmaml = [
    torch.cuda.FloatTensor(nh, nx).uniform_(-1., 1.).requires_grad_(),
   

]

params_krmaml_cos = [
    torch.cuda.FloatTensor(nh, nx).uniform_(-1., 1.).requires_grad_(),
    

]

n_inner_loop=1

params_metasgd = [
    torch.cuda.FloatTensor(nh, nx).uniform_(-1., 1.).requires_grad_(),
    

]

# Learning rate for Meta-SGD
alp_metasgd = [
    torch.cuda.FloatTensor(nh, nx).uniform_(1e-2, 1e-2).requires_grad_(),
    

]



# kernel regression coefficients for TANML: W is the same as Psi in the paper
# Gaussian kernel
W_krmaml = []
for i in range(len(params_krmaml)):
    s = len(params_krmaml[i].view(-1))
    W_krmaml.append((torch.cuda.FloatTensor(s, n_tasks).uniform_(-1./math.sqrt(s*n_tasks),1./math.sqrt(s*n_tasks)).requires_grad_()))


#Cosine kernel
W_krmaml_cos = []
for i in range(len(params_krmaml)):
    s = len(params_krmaml_cos[i].view(-1))
    W_krmaml_cos.append((torch.cuda.FloatTensor(s, n_tasks).uniform_(-1./math.sqrt(s*n_tasks),1./math.sqrt(s*n_tasks)).requires_grad_()))
   

#MAML Meta train

def maml(x, y, n_meta_train, params, alpha, c):

    
    plt.clf()
    colors = cmap(np.linspace(0, 1, (n_tasks)))
    for it in range(n_meta_train):

        new_params = params
        # print(new_params[0][0:10])
        loss_full = torch.cuda.FloatTensor(1).zero_()
        loss2_full = torch.cuda.FloatTensor(1).zero_()
        
        for k in range(n_inner_loop):
            t_e = 0
            for tasks in range(n_tasks):
                color=colors[tasks]
                new_params = params
                #tempx = x[tasks]
                # print(tempx)
                #tempy = y[tasks]
                f = net(x[tasks], new_params)
                loss = F.mse_loss(f, y[tasks])
                loss_full = loss_full + loss
                # create_graph=True because computing grads here is part of the forward pass.
                # We want to differentiate through the SGD update steps and get higher order
                # derivatives in the backward pass.
                grads = torch.autograd.grad(loss, new_params, create_graph=False)
                new_params = [(params[i] - alpha * grads[i]) for i in range(len(params))]

                
                v_tempf = net(v_x[tasks], new_params)
                loss2 = F.mse_loss(v_tempf, v_y[tasks])
                loss2_full = loss2_full + loss2
                t_e = t_e + torch.norm(v_y[tasks], p=2)**2/len(v_y[tasks])

        loss_tr = loss2_full


        dis = 10 * torch.log10(loss_tr / t_e)
        nmse_train=dis.clone().detach()
        if c % 50 == 0: print('MAML train Iteration %d -- Outer Loss: %.4f' % (c, dis))
    #     # print(*alpha,sep=',')
    del loss, loss2, loss_full, loss_tr, new_params, dis, v_tempf
    torch.cuda.empty_cache()
    return params, nmse_train, loss2_full, (c + 1)

#Meta-SGD Meta train

def metasgd(x, y, n_meta_train, params, alp_metasgd, c):

    
    plt.clf()
    colors = cmap(np.linspace(0, 1, (n_tasks)))
    for it in range(n_meta_train):

        new_params = params
        # print(new_params[0][0:10])
        loss_full = torch.cuda.FloatTensor(1).zero_()
        loss2_full = torch.cuda.FloatTensor(1).zero_()
        
        for k in range(n_inner_loop):
            t_e = 0
            for tasks in range(n_tasks):
                color=colors[tasks]
                new_params = params
                #tempx = x[tasks]
                # print(tempx)
                #tempy = y[tasks]
                f = net(x[tasks], new_params)
                loss = F.mse_loss(f, y[tasks])
                loss_full = loss_full + loss
                # create_graph=True because computing grads here is part of the forward pass.
                # We want to differentiate through the SGD update steps and get higher order
                # derivatives in the backward pass.
                grads = torch.autograd.grad(loss, new_params, retain_graph=False)
                new_params = [(params[i] - alp_metasgd[i]*grads[i]) for i in range(len(params))]
                #print(grads)

                v_tempf = net(v_x[tasks], new_params)
                loss2 = F.mse_loss(v_tempf, v_y[tasks])
                loss2_full = loss2_full + loss2
                t_e = t_e + torch.norm(v_y[tasks], p=2)**2/len(v_y[tasks])

                
        loss_tr = loss2_full


        dis = 10 * torch.log10(loss_tr / t_e)
        nmse_train=dis.clone().detach()
        if c % 50 == 0: print('MetaSGD train Iteration %d -- Outer Loss: %.4f' % (c, dis))
    #     # print(*alpha,sep=',')
    del loss, loss2, loss_full, loss_tr, new_params, dis, v_tempf
    torch.cuda.empty_cache()
    return params, alp_metasgd, nmse_train, loss2_full, (c + 1)


## TANML (My method): Meta Trainng
def krmaml(x, y, n_meta_train, W, params, c,kernel,reg):
  
    plt.clf()
    colors = cmap(np.linspace(0, 1, (n_tasks)))
    for it in range(n_meta_train):
        new_params = params
     
        loss_full = torch.cuda.FloatTensor(1).zero_()
      
        Z_param_train = []
        new_params = params
        with torch.no_grad(): grads=[(params) for i in range(n_tasks)]
        K = torch.cuda.FloatTensor(len(params), n_tasks, n_tasks).zero_()
        for tasks in range(n_tasks):
            color=colors[tasks]
            
            f = net(x[tasks], new_params)
            #print(f)
            loss = F.mse_loss(f, y[tasks])
            loss_full = loss_full + loss
            grads[tasks] = torch.autograd.grad(loss, new_params, create_graph=False)
                
        for i in range(len(params)):

            Z = []
            for tasks in range(n_tasks):
         
            
                z = torch.stack([new_params[i], (grads[tasks][i])]).view(-1) # almost twice slower than MAML
               # z = torch.stack([(grads[tasks][i])]).view(-1) # similarly fast as MAML but performs worse than the other
                
                #z.requires_grad_()
                Z.append(z)
            Z=torch.stack(Z)
            #print(Z)

            Z_param_train.append(Z)

            # Z.requires_grad_()

            
            if kernel=="cosine": K[i, :,:] = cos_dist(Z,Z)
            elif kernel == "gaussian": K[i, :,:] =torch.exp(-dist(Z,Z)/(sigma**2))
        #print(K)

        loss2_full = torch.cuda.FloatTensor(1).zero_()
        t_e = 0
        for tasks in range(n_tasks):
            color=colors[tasks]
            new_params_vec = [(torch.matmul(W[i], K[i, :, tasks])) for i in range(len(params))]
            # with torch.no_grad():
            new_params_t = [(new_params_vec[i].reshape(params[i].size())) for i in range(len(params))]
            
            v_tempf = net(v_x[tasks], new_params_t)
            #print(v_tempf)
           
            loss2 = F.mse_loss(net(v_x[tasks], new_params_t), v_y[tasks])
            loss2_full = loss2_full + loss2
            t_e = t_e + torch.norm(v_y[tasks], p=2)**2/len(v_y[tasks])
           
        loss_tr = loss2_full
        loss_W = torch.cuda.FloatTensor(1).zero_()
        for i in range(len(W)):
            loss_W = loss_W + reg * torch.trace(torch.matmul(W[i],torch.matmul(K[i,:,:],W[i].t())))
        loss2_full = loss2_full + loss_W

        dis = 10 * torch.log10(loss_tr / t_e)
        nmse_train=dis.clone().detach()
        if c % 50 == 0: print('KRMAML train Iteration %d -- Outer Loss: %.4f' % (c, dis))
     
    del loss, loss2, loss_full, loss_W, loss_tr, new_params, new_params_vec, z, dis, v_tempf
    torch.cuda.empty_cache()
    return params, nmse_train, Z_param_train, K, loss2_full, (c + 1)

######## TEST
# MAML
def maml_test(t_x_, t_y_, v_x_, v_y_, params, n_tasks_test, A_):
    t_params = params
    loss_full = torch.cuda.FloatTensor(1).zero_()
    t_loss = torch.cuda.FloatTensor(1).zero_()
    t_e = torch.cuda.FloatTensor(1).zero_()
    cmap = plt.get_cmap('jet')
    colors = cmap(np.linspace(0.1, .9, (n_tasks_test)))
    plt.clf()
    for tasks in range(n_tasks_test):
        color = colors[tasks]

        t_params = params
       
        f = net(t_x_[tasks], t_params)
        loss = F.mse_loss(f, t_y_[tasks])
        loss_full = loss_full + loss
        grads = torch.autograd.grad(loss, t_params, create_graph=False)
        new_params = [(params[i] - alpha * grads[i]) for i in range(len(params))]


        v_tempf = net(v_x_[tasks], t_params)
        loss2 = F.mse_loss(v_tempf, v_y_[tasks])
        t_loss = t_loss + loss2
        t_e = t_e + torch.norm(v_y_[tasks], p=2)**2/len(v_y_[tasks])

        
        dis = 10*torch.log10((t_loss / t_e))
        # nmse_test.append(dis)
        nmse_test=dis.clone().detach()
    # print('MAML Test Loss: %.4f' % (dis))
    del loss, loss2, t_loss, t_params, new_params, dis, v_tempf
    torch.cuda.empty_cache()
    return params, nmse_test

# TANML (My)
def krmaml_test(t_x_, t_y_, v_x_, v_y_, params, n_tasks_test, W, Z_param_train, A_,kernel):
    cmap = plt.get_cmap('jet')
    colors = cmap(np.linspace(0.1, .9, (n_tasks_test)))
    
    # K.requires_grad_()
    new_params = params
    loss_full = torch.cuda.FloatTensor(1).zero_()
      
    Z_test = []
    new_params = params
    with torch.no_grad(): grads=[(params) for i in range(n_tasks_test)]
    K_test = torch.cuda.FloatTensor(len(params), n_tasks, n_tasks_test).zero_()
    for tasks in range(n_tasks_test):
        color=colors[tasks]
        
        f = net(t_x_[tasks], new_params)
        #print(f)
        loss = F.mse_loss(f, t_y_[tasks])
        loss_full = loss_full + loss
        grads[tasks] = torch.autograd.grad(loss, new_params, create_graph=False)
                
        for i in range(len(params)):
            Z = Z_param_train[i]
            Z_test = []
            for tasks in range(n_tasks_test):
         
            
                with torch.no_grad():z_test = torch.stack([new_params[i], (grads[tasks][i])]).view(-1)
                #with torch.no_grad():z_test = torch.stack([(grads[tasks][i])]).view(-1)
                
                #z_test.requires_grad_()
                with torch.no_grad():Z_test.append(z_test)
            with torch.no_grad():Z_test=torch.stack(Z_test)
            #print(Z)

            

            
           
            with torch.no_grad():
              if kernel=="cosine": K_test[i, :,:] = cos_dist(Z,Z_test)
              elif kernel=="g_rff": K_test[i, :,:] = dist_rff(Z,Z_test,Omega[i])
              elif kernel == "gaussian": K_test[i, :,:]=torch.exp(-dist(Z,Z_test)/(sigma**2))
                    # print(K)    
   
    with torch.no_grad():t_loss = torch.cuda.FloatTensor(1).zero_()
    with torch.no_grad():t_e = torch.cuda.FloatTensor(1).zero_()

    colors = cmap(np.linspace(0.1, .9, (n_tasks_test)))
    for tasks in range(n_tasks_test):
        color = colors[tasks]

        with torch.no_grad():new_params_vec = [(torch.matmul(W[i], K_test[i, :, tasks])) for i in range(len(params))]
        # with torch.no_grad():
        with torch.no_grad():new_params = [(new_params_vec[i].reshape(params[i].size())) for i in range(len(params))]

        #v_tempx = v_x[tasks]
        #v_tempy = v_y[tasks]
        with torch.no_grad():v_tempf = net(v_x_[tasks], new_params)
        with torch.no_grad():loss2 = F.mse_loss(v_tempf, v_y_[tasks])
        with torch.no_grad():t_loss = t_loss + loss2
        with torch.no_grad():t_e = t_e + torch.norm(v_y_[tasks], p=2)**2/len(v_y_[tasks])

        with torch.no_grad():dis = 10*torch.log10((t_loss / t_e))
        #nmse_test = dis.item()
        with torch.no_grad():nmse_test=dis.clone().detach()
  
    del loss, loss2, t_loss, Z, new_params, new_params_vec, dis, Z_test, z_test, v_tempf
    torch.cuda.empty_cache()
    return params, nmse_test, K_test

# Meta-SGD
def metasgd_test(t_x_, t_y_, v_x_, v_y_, params, n_tasks_test, A_):
    t_params = params
    loss_full = torch.cuda.FloatTensor(1).zero_()
    t_loss = torch.cuda.FloatTensor(1).zero_()
    t_e = torch.cuda.FloatTensor(1).zero_()
    cmap = plt.get_cmap('jet')
    colors = cmap(np.linspace(0.1, .9, (n_tasks_test)))
    plt.clf()
    for tasks in range(n_tasks_test):
        color = colors[tasks]

        t_params = params
        
        f = net(t_x_[tasks], t_params)
        loss = F.mse_loss(f, t_y_[tasks])
        loss_full = loss_full + loss
        grads = torch.autograd.grad(loss, t_params)
        new_params = [(params[i] - alp_metasgd[i] * grads[i]) for i in range(len(params))]

        v_tempf = net(v_x_[tasks], new_params)
        loss2 = F.mse_loss(v_tempf, v_y_[tasks])
        t_loss = t_loss + loss2
        t_e = t_e + torch.norm(v_y_[tasks], p=2)**2/len(v_y_[tasks])

        
        dis = 10*torch.log10((t_loss / t_e))
        # nmse_test.append(dis)
        nmse_test=dis.clone().detach()
    # print('MAML Test Loss: %.4f' % (dis))
    del loss, loss2, t_loss, t_params, new_params, dis, v_tempf
    torch.cuda.empty_cache()
    return params, alp_metasgd,nmse_test


######## Main
x = []
y = []
v_x = []
v_y = []
t = time.time()
A = []
nmse_train = []
loss2_full_maml = torch.cuda.FloatTensor(1).zero_()
loss2_full_krmaml = torch.cuda.FloatTensor(1).zero_()

# Generating training and testing sinusoids
w1= 4+torch.cuda.FloatTensor(1, nx).normal_()
w2= -4+torch.cuda.FloatTensor(1, nx).normal_()
# Training tasks
for tasks in range(n_tasks):
    #Train data
     
    tempx = F.normalize(torch.cuda.FloatTensor(kshot, nx).uniform_()) 
    v_tempx = F.normalize(torch.cuda.FloatTensor(kshot, nx).uniform_()) 
    
    if tasks%(1)==0: 
        tempy=F.linear(tempx, w1) + torch.cuda.FloatTensor(kshot, 1).uniform_() 
        v_tempy=F.linear(tempx, w1) + torch.cuda.FloatTensor(kshot, 1).uniform_()

    if tasks%(2)==0: 
        tempy=F.linear(tempx, w2) + torch.cuda.FloatTensor(kshot, 1).uniform_()
        v_tempy=F.linear(tempx, w2) + torch.cuda.FloatTensor(kshot, 1).uniform_()
   
        
    x.append(tempx)
    y.append(tempy)
    
    #Validation or test datax
        
    v_x.append(v_tempx)
    v_y.append(v_tempy)

    del tempx,tempy,v_tempx,v_tempy

#Test tasks
t_x_ = []
t_y_ = []
A_ = []
v_x_ = []
v_y_ = []
for tasks in range(n_tasks_test):
         
    tempx = F.normalize(torch.cuda.FloatTensor(kshot, nx).uniform_()) 
    v_tempx = F.normalize(torch.cuda.FloatTensor(kshot, nx).uniform_()) 
    
    if tasks%(1)==0: 
        tempy=F.linear(tempx, w1) + torch.cuda.FloatTensor(kshot, 1).uniform_() 
        v_tempy=F.linear(tempx, w1) + torch.cuda.FloatTensor(kshot, 1).uniform_()

    if tasks%(2)==0: 
        tempy=F.linear(tempx, w2) + torch.cuda.FloatTensor(kshot, 1).uniform_()
        v_tempy=F.linear(tempx, w2) + torch.cuda.FloatTensor(kshot, 1).uniform_()
   
        
    t_x_.append(tempx)
    t_y_.append(tempy)
    
    #Validation or test datax
        
    v_x_.append(v_tempx)
    v_y_.append(v_tempy)

    del tempx,tempy,v_tempx,v_tempy



NMSE_train_maml = []
NMSE_train_krmaml = []
NMSE_train_krmaml_cos = []
NMSE_test_maml = []
NMSE_test_krmaml = []
NMSE_test_krmaml_cos = []
test_axis_x=[]

NMSE_train_maml = []
NMSE_train_krmaml = []
NMSE_train_krmaml_cos = []
NMSE_test_maml = []
NMSE_test_krmaml = []
NMSE_test_krmaml_cos = []
test_axis_x=[]

NMSE_train_metasgd=[]
NMSE_test_metasgd=[]

# defining optimizers
opt_metasgd = torch.optim.Adam([
    {'params': params_metasgd, },
    {'params': alp_metasgd, 'lr': 1e-6}
], lr=lr_maml)
loss2_full_metasgd = torch.cuda.FloatTensor(1).zero_()

opt_maml = torch.optim.Adam(params_maml, lr=lr_maml)
opt_krmaml = torch.optim.Adam([
    {'params': params_krmaml, },
    {'params': W_krmaml, 'lr': lr_W}
], lr=lr_krmaml)
opt_krmaml_cos = torch.optim.Adam([
    {'params': params_krmaml_cos, },
    {'params': W_krmaml_cos, 'lr': lr_W_cos}
], lr=lr_krmaml_cos)


if device == torch.device('cuda'):
    torch.cuda.synchronize(device=device)
t1=time.time() 
for tstep in range(n_meta_train):
    
    opt_maml.zero_grad()
    opt_krmaml.zero_grad()
    opt_krmaml_cos.zero_grad()
    
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t1=time.time()
    [params_krmaml, nmse_train_krmaml, Z_param_train, K, loss2_full_krmaml, c] = krmaml(x, y, 1, W_krmaml,
                                                                                         params_krmaml, 
                                                                                         tstep,kernel_1,reg)
    #
    loss2_full_krmaml.backward()
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t2=time.time()
    #print('kr1 time:',(t2-t1))
    opt_krmaml.step()

    

    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t1=time.time()
    [params_krmaml_cos, nmse_train_krmaml_cos, Z_param_train_cos, K_cos, loss2_full_krmaml_cos, c] = krmaml(x, y, 1, W_krmaml_cos,
                                                                                         params_krmaml_cos, 
                                                                                         tstep,kernel_2,reg_cos)
    #
    loss2_full_krmaml_cos.backward()
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t2=time.time()

    opt_krmaml_cos.step()

    
    #print('kr2 time:',(t2-t1))
    
   
    NMSE_train_krmaml.append(nmse_train_krmaml)
    NMSE_train_krmaml_cos.append(nmse_train_krmaml_cos)

    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t1=time.time()
    [params_maml, nmse_train_maml, loss2_full_maml, c] = maml(x, y, 1, params_maml, alpha, tstep)
 
    loss2_full_maml.backward()
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t2=time.time()
    #print('maml time:',(t2-t1))
    opt_maml.step()
    NMSE_train_maml.append(nmse_train_maml)


    opt_metasgd.zero_grad()
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t1=time.time()
    [params_metasgd, alp_metasgd, nmse_train_metasgd, loss2_full_metasgd, c] = metasgd(x, y, 1, params_metasgd, alp_metasgd, tstep)
    loss2_full_metasgd.backward()
    if device == torch.device('cuda'): torch.cuda.synchronize(device=device) 
    t2=time.time()
    #print('metasgd time:',(t2-t1))
    NMSE_train_metasgd.append(nmse_train_metasgd)
    opt_metasgd.step()

    
   
    # On every t_step iterations, compute the TEST loss and
    # plot the TRAIN and TEST loss
    if tstep % test_step== 0:
      plt.clf()
      plt.plot(NMSE_train_maml, label='maml')
      plt.plot(NMSE_train_krmaml, label='krmaml-gaussian')
      plt.plot(NMSE_train_krmaml_cos, label='krmaml-cosine')
      plt.plot(NMSE_train_metasgd, label='metasgd')
      plt.legend()
    
      plt.legend()
      fig_name='/content/drive/My Drive/Colab Notebooks/Ex_lr/train_nmse_'+str(varying_amp)+fig_id+'.jpg'
      plt.savefig(fig_name)  

    
     
      [params_krmaml, nmse_test_krmaml_, K_test_] = krmaml_test(t_x_, t_y_, v_x_, v_y_, params_krmaml, n_tasks_test, W_krmaml, Z_param_train, A_,kernel_1)
      [params_krmaml_cos, nmse_test_krmaml_cos, K_test_cos] = krmaml_test(t_x_, t_y_, v_x_, v_y_, params_krmaml_cos, n_tasks_test, W_krmaml_cos, Z_param_train_cos, A_,kernel_2)
      [params_maml, nmse_test_maml_] = maml_test(t_x_, t_y_, v_x_, v_y_, params_maml, n_tasks_test, A_)
      [params_metasgd,alp_metasgd, nmse_test_metasgd_] = metasgd_test(t_x_, t_y_, v_x_, v_y_, params_metasgd, n_tasks_test, A_)
      
      NMSE_test_metasgd.append(nmse_test_metasgd_)
      NMSE_test_maml.append(nmse_test_maml_)
      NMSE_test_krmaml.append(nmse_test_krmaml_)
      NMSE_test_krmaml_cos.append(nmse_test_krmaml_cos)

      test_axis_x.append(tstep)
      plt.clf()
      plt.plot(test_axis_x,NMSE_test_maml, label='maml')
      plt.plot(test_axis_x,NMSE_test_krmaml,label='krmaml-gaussian')
      plt.plot(test_axis_x,NMSE_test_krmaml_cos,label='krmaml-cosine')
      plt.plot(test_axis_x,NMSE_test_metasgd,label='metasgd')
      
      plt.legend()
      fig_name='/content/drive/My Drive/Colab Notebooks/Ex_lr/test_nmse_'+str(varying_amp)+fig_id+'.jpg'
      plt.savefig(fig_name)
      #t2=time.time()  
      #if device == torch.device('cuda'):
      #  torch.cuda.synchronize(device=device)
      #t2=time.time() 
      #print('device time:',(t2-t1))
      filepath='/content/drive/My Drive/Colab Notebooks/Ex_lr/params'+str(varying_amp)+fig_id+'.pkl'
      dill.dump_session(filepath) # Saves session with all variables
      #print('Epoch time:',(t2-t1))
      #t1=t2
    torch.cuda.empty_cache()
      
     

KRMAML train Iteration 0 -- Outer Loss: 0.0310
KRMAML train Iteration 0 -- Outer Loss: 0.0549
MAML train Iteration 0 -- Outer Loss: -0.6757
MetaSGD train Iteration 0 -- Outer Loss: -0.1101
KRMAML train Iteration 50 -- Outer Loss: -0.6768
KRMAML train Iteration 50 -- Outer Loss: -3.5122
MAML train Iteration 50 -- Outer Loss: -0.7158
MetaSGD train Iteration 50 -- Outer Loss: -0.1535
KRMAML train Iteration 100 -- Outer Loss: -1.4140
KRMAML train Iteration 100 -- Outer Loss: -7.9409
MAML train Iteration 100 -- Outer Loss: -0.7518
MetaSGD train Iteration 100 -- Outer Loss: -0.1908
KRMAML train Iteration 150 -- Outer Loss: -2.1777
KRMAML train Iteration 150 -- Outer Loss: -13.0361
MAML train Iteration 150 -- Outer Loss: -0.7842
MetaSGD train Iteration 150 -- Outer Loss: -0.2243
KRMAML train Iteration 200 -- Outer Loss: -2.9678
KRMAML train Iteration 200 -- Outer Loss: -17.9612
MAML train Iteration 200 -- Outer Loss: -0.8134
MetaSGD train Iteration 200 -- Outer Loss: -0.2546
KRMAML train Iter

In [3]:
print(torch.cuda.FloatTensor(1, nx).normal_())

tensor([[-0.6763, -0.3089,  0.7884,  1.4584,  0.8911,  0.9780, -1.2378,  1.5669,
         -1.0677, -0.1777,  1.6383, -1.5633,  0.9397,  0.8877,  2.2099, -0.8674,
          0.2507,  2.5102,  1.9004, -1.5710, -0.5280,  0.1271,  0.0906,  0.6183,
          1.6802,  0.2316,  0.5417, -2.3063,  1.7949,  0.6272,  0.6556, -0.0700]],
       device='cuda:0')
